In [15]:
from datetime import date, datetime
from dataclasses import dataclass
from enum import Enum, auto
import pandas as pd
import re
from typing import Dict, List


class TxType(Enum):
    DEPO = auto()
    WITH = auto()


class TxSummary(Enum):
    CLIENT_CONSIDERATION = auto()
    DIVIDEND = auto()

@dataclass
class TxMarketDetails:
    security: str
    quantity: int
    value_usd_cents: float
    pnl_usd_cents: int


@dataclass
class HistoricalTx:
    summary: TxSummary
    tx_type: TxType
    market_details: TxMarketDetails
    date_utc: datetime


@dataclass
class CapitalGains:
    source: str
    quantity: int
    value_usd_cents: int
    when: datetime


def parse_historical_txs(tx_history_file_path: str) -> List[HistoricalTx]:
    historical_txs: List[HistoricalTx] = []

    tx_history: pd.DataFrame = pd.read_csv(tx_history_file_path)

    for index, row in tx_history.iterrows():
        tx_summary: str = row['Summary']

        if tx_summary not in ['Client Consideration', 'Dividend']:
            continue

        summary: TxSummary = parse_tx_summary(tx_summary)
        tx_type: TxType = parse_tx_type(row['Transaction type'])
        date_utc: datetime = datetime.strptime(row['DateUtc'], '%Y-%m-%dT%H:%M:%S')

        if summary is TxSummary.CLIENT_CONSIDERATION:
            market_details: TxMarketDetails = parse_clcons_market_name(row['MarketName'], tx_type, date_utc)
        elif summary is TxSummary.DIVIDEND:
            market_details: TxMarketDetails = parse_div_market_name(row['MarketName'])

        historical_txs.append(HistoricalTx(summary, tx_type, market_details, date_utc))
    
    return historical_txs


def parse_tx_type(tx_type: str) -> TxType:
    if tx_type == 'DEPO':
        return TxType.DEPO
    if tx_type == 'WITH':
        return TxType.WITH
    raise ValueError(f"An unsupported transaction type was provided as input: {tx_type}.")


def parse_tx_summary(tx_summary: str) -> TxSummary:
    if tx_summary == 'Client Consideration':
        return TxSummary.CLIENT_CONSIDERATION
    elif tx_summary == 'Dividend':
        return TxSummary.DIVIDEND
    raise ValueError(f"An unsupported transaction summary was provided as input: {tx_summary}.")


def parse_div_market_name(market_name: str) -> TxMarketDetails:
    div_match: re.Match = re.search("(.*) DIVIDEND ([0-9]+)@([0-9.]+)", market_name)

    if not div_match:
        raise ValueError(f"Could not successfully parse a dividend market name: {market_name}.")

    security: str = div_match.group(1)
    quantity: int = int(div_match.group(2))
    value_usd_cents: int = float(div_match.group(3)) * 100
    
    pnl_usd_cents: int = round(quantity * value_usd_cents)

    return TxMarketDetails(security, quantity, value_usd_cents, pnl_usd_cents)


def rename_security(security: str) -> str:
    if security == "Facebook Inc (All Sessions)":
        return "Meta Platforms Inc (All Sessions)"
    if security == "NVIDIA Corporation":
        return "NVIDIA Corp (All Sessions)"
    return security


def update_quantity_due_to_splits(security: str, day: datetime, quantity: int) -> int:
    if security == "NVIDIA Corp (All Sessions)" and day.date() < date(2021, 9, 27):
        return quantity * 4
    return quantity


def update_value_usd_cents_due_to_splits(security: str, day: datetime, value_usd_cents: float) -> float:
    if security == "NVIDIA Corp (All Sessions)" and day.date() < date(2021, 9, 27):
        return value_usd_cents / 4
    return value_usd_cents


def parse_clcons_market_name(market_name: str, tx_type: TxType, day: datetime) -> TxMarketDetails:
    clcons_match: re.Match = re.search("(.*) CONS ([0-9]+)@([0-9.]+)", market_name)
    
    if not clcons_match:
        raise ValueError(f"Could not successfully parse a client consideration market name: {market_name}.")

    security: str = clcons_match.group(1)
    renamed_security = rename_security(security)
    quantity: int = int(clcons_match.group(2))
    updated_quantity = update_quantity_due_to_splits(renamed_security, day, quantity)  
    value_usd_cents: float = float(clcons_match.group(3))
    updated_value_usd_cents = update_value_usd_cents_due_to_splits(renamed_security, day, value_usd_cents)

    pnl_sign: int = -1 if tx_type is TxType.WITH else 1
    pnl_usd_cents: int = round(quantity * value_usd_cents) * pnl_sign

    return TxMarketDetails(renamed_security, updated_quantity, updated_value_usd_cents, pnl_usd_cents)


def compute_capital_gains(historical_txs: List[HistoricalTx], start: datetime, stop: datetime) -> List[CapitalGains]:
    capital_gains: List[CapitalGains] = []
    
    security_txs: Dict[str, List[HistoricalTx]] = {}

    for tx in historical_txs:
        if tx.summary is TxSummary.DIVIDEND:
            if tx.date_utc >= start and tx.date_utc <= stop:
                capital_gains.append(CapitalGains(f"IG {tx.market_details.security} dividends", tx.market_details.quantity, tx.market_details.pnl_usd_cents, tx.date_utc))
        elif tx.summary is TxSummary.CLIENT_CONSIDERATION:
            if tx.tx_type == TxType.DEPO:
                if tx.market_details.security not in security_txs:
                    raise RuntimeError(f"No unmatched buy transactions exist in the transaction history for security {tx.market_details.security} before {tx.date_utc}.")
                
                value_usd_cents: int = 0
                quantity_to_sell: int = tx.market_details.quantity
                unmatched_buy_txs: List[HistoricalTx] = security_txs[tx.market_details.security]
                
                while quantity_to_sell > 0:
                    if not unmatched_buy_txs:
                        raise RuntimeError(f"No more unmatched buy transactions exist in the transaction history for security {tx.market_details.security} before {tx.date_utc}.")

                    buy_tx: HistoricalTx = unmatched_buy_txs[0]
                    quantity_to_match: int = min(quantity_to_sell, buy_tx.market_details.quantity)

                    value_usd_cents += quantity_to_match * (tx.market_details.value_usd_cents - buy_tx.market_details.value_usd_cents)
                    quantity_to_sell -= quantity_to_match
                    
                    if quantity_to_match == buy_tx.market_details.quantity:
                        unmatched_buy_txs.pop(0)
                
                security_txs[tx.market_details.security] = unmatched_buy_txs
                
                if value_usd_cents >= 0 and tx.date_utc >= start and tx.date_utc <= stop:
                    capital_gains.append(CapitalGains(f"IG {tx.market_details.security} shares sale", tx.market_details.quantity, value_usd_cents, tx.date_utc))
            elif tx.tx_type == TxType.WITH:
                if tx.market_details.security in security_txs:
                    security_txs[tx.market_details.security].append(tx)
                else:
                    security_txs[tx.market_details.security] = [tx]

    return capital_gains


# TODO: Update the values of the variables given immediately below as needed
tx_history_file_path: str = r"<path-to-dir-containing-ig-tx-hist-files>\TransactionHistory-(31-08-2019)-(13-01-2024).csv"
financial_year: int = 2022
start_datetime: datetime = datetime(year=financial_year, month=4, day=6)
stop_datetime: datetime = datetime(year=financial_year+1, month=4, day=5)
    
# Load historical transactions in chronological order
historical_txs: List[HistoricalTx] = parse_historical_txs(tx_history_file_path)
historical_txs.sort(key=lambda tx: tx.date_utc)

# Print capital gains
capital_gains: List[CapitalGains] = compute_capital_gains(historical_txs, start_datetime, stop_datetime)

capital_gains

[]

In [16]:
# Print capital gains
for gain in capital_gains:
    print(f'{gain.source},{gain.quantity},{gain.value_usd_cents/100},,{gain.when.strftime("%d/%m/%Y")}')